# CAFA 6 Protein Function Prediction - Exploratory Data Analysis

This notebook explores the training data to understand:
- Distribution of GO terms per protein
- GO term frequency distribution
- Sequence length distribution
- GO term aspects (Cellular Component, Molecular Function, Biological Process)
- Taxonomic distribution

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent))

from src.data import load_sequences_from_fasta, load_go_terms_long_format, build_samples

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## Load Data

In [ ]:
# Define paths
data_dir = Path.cwd().parent / "data" / "raw" / "cafa-6-protein-function-prediction" / "Train"
fasta_path = data_dir / "train_sequences.fasta"
terms_path = data_dir / "train_terms.tsv"
taxonomy_path = data_dir / "train_taxonomy.tsv"

print(f"Loading sequences from {fasta_path}")
sequences = load_sequences_from_fasta(fasta_path)
print(f"Loaded {len(sequences)} protein sequences")

print(f"\nLoading GO terms from {terms_path}")
annotations = load_go_terms_long_format(terms_path)
print(f"Loaded annotations for {len(annotations)} proteins")

print(f"\nBuilding samples...")
samples = build_samples(sequences, annotations)
print(f"Built {len(samples)} samples")

## Load GO Terms with Aspects

In [ ]:
# Load raw terms data to get aspects
terms_df = pd.read_csv(terms_path, sep='\t')
print(f"Total GO term annotations: {len(terms_df)}")
print(f"\nFirst few rows:")
terms_df.head(10)

In [ ]:
# Aspect distribution
aspect_counts = terms_df['aspect'].value_counts()
print("GO Term Aspect Distribution:")
print(aspect_counts)
print(f"\nC = Cellular Component")
print(f"F = Molecular Function")
print(f"P = Biological Process")

## Sequence Length Analysis

In [ ]:
# Sequence lengths
seq_lengths = [len(sample.sequence) for sample in samples]

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram
axes[0].hist(seq_lengths, bins=50, edgecolor='black')
axes[0].set_xlabel('Sequence Length')
axes[0].set_ylabel('Count')
axes[0].set_title('Distribution of Protein Sequence Lengths')
axes[0].axvline(np.median(seq_lengths), color='red', linestyle='--', label=f'Median: {np.median(seq_lengths):.0f}')
axes[0].legend()

# Box plot
axes[1].boxplot(seq_lengths, vert=True)
axes[1].set_ylabel('Sequence Length')
axes[1].set_title('Sequence Length Box Plot')

plt.tight_layout()
plt.show()

print(f"Sequence Length Statistics:")
print(f"  Min: {min(seq_lengths)}")
print(f"  Max: {max(seq_lengths)}")
print(f"  Mean: {np.mean(seq_lengths):.2f}")
print(f"  Median: {np.median(seq_lengths):.2f}")
print(f"  Std: {np.std(seq_lengths):.2f}")

## GO Terms per Protein

In [ ]:
# GO terms per protein
terms_per_protein = [len(sample.go_terms) for sample in samples]

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histogram
axes[0].hist(terms_per_protein, bins=50, edgecolor='black')
axes[0].set_xlabel('Number of GO Terms')
axes[0].set_ylabel('Count')
axes[0].set_title('Distribution of GO Terms per Protein')
axes[0].axvline(np.median(terms_per_protein), color='red', linestyle='--', label=f'Median: {np.median(terms_per_protein):.0f}')
axes[0].legend()

# Log scale
axes[1].hist(terms_per_protein, bins=50, edgecolor='black')
axes[1].set_xlabel('Number of GO Terms')
axes[1].set_ylabel('Count (log scale)')
axes[1].set_yscale('log')
axes[1].set_title('Distribution of GO Terms per Protein (Log Scale)')

plt.tight_layout()
plt.show()

print(f"GO Terms per Protein Statistics:")
print(f"  Min: {min(terms_per_protein)}")
print(f"  Max: {max(terms_per_protein)}")
print(f"  Mean: {np.mean(terms_per_protein):.2f}")
print(f"  Median: {np.median(terms_per_protein):.2f}")
print(f"  Std: {np.std(terms_per_protein):.2f}")

## GO Term Frequency

In [ ]:
# Count GO term frequencies
all_terms = []
for sample in samples:
    all_terms.extend(sample.go_terms)

term_counts = Counter(all_terms)
print(f"Total unique GO terms: {len(term_counts)}")
print(f"Total GO term annotations: {len(all_terms)}")
print(f"\nTop 20 most frequent GO terms:")
for term, count in term_counts.most_common(20):
    print(f"  {term}: {count}")

In [ ]:
# Plot term frequency distribution
frequencies = list(term_counts.values())

plt.figure(figsize=(12, 5))
plt.hist(frequencies, bins=100, edgecolor='black')
plt.xlabel('Frequency')
plt.ylabel('Number of GO Terms')
plt.title('GO Term Frequency Distribution')
plt.yscale('log')
plt.xscale('log')
plt.grid(True, alpha=0.3)
plt.show()

print(f"\nFrequency Statistics:")
print(f"  Min: {min(frequencies)}")
print(f"  Max: {max(frequencies)}")
print(f"  Mean: {np.mean(frequencies):.2f}")
print(f"  Median: {np.median(frequencies):.2f}")

## Summary Statistics

In [ ]:
print("=" * 60)
print("CAFA 6 Dataset Summary")
print("=" * 60)
print(f"Total proteins: {len(samples)}")
print(f"Total unique GO terms: {len(term_counts)}")
print(f"Total annotations: {len(all_terms)}")
print(f"Average GO terms per protein: {np.mean(terms_per_protein):.2f}")
print(f"Average sequence length: {np.mean(seq_lengths):.2f}")
print(f"\nGO Aspect Distribution:")
for aspect, count in aspect_counts.items():
    print(f"  {aspect}: {count} ({count/len(terms_df)*100:.1f}%)")
print("=" * 60)